In [1]:
# syft absolute
import syft as sy
from syft.types.syft_object import SyftObject

In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=5f2b3a35fe5f43b2ac56f60c9a39b6c6 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=fce552471b4f4f348d5a66d33fae1e71 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

# create datasets

In [4]:
# third party
import numpy as np

mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=private_low.shape,
            mock_is_real=True,
        )
    ],
)

client_low.upload_dataset(dataset_low)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.68it/s]


SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [5]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    id=dataset_low.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.96it/s]


Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [6]:
data_low = client_low.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/99c248b9e57f4d64b5086d03602fb932.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=ff5630f4e060467da9bd3da47ca690ed in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_4908: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_4908
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [7]:
client_low.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [8]:
request = client_low.requests[0]

In [9]:
code = client_low.code[0]

TEO HERE

In [10]:
items_to_sync = [request, code]

- What do we do with properties here?
  - just set them to None

In [11]:
def get_stash_for_item(item, node):
    services = list(node.python_node.service_path_map.values())

    all_stashes = dict()
    for serv in services:
        if (_stash := getattr(serv, "stash", None)) is not None:
            all_stashes[_stash.object_type] = _stash

    stash = all_stashes.get(type(item), None)
    return stash

In [12]:
# job.node_uid

In [13]:
node_high.python_node.id

<UID: 8212e6797fde4c3fba4fc53ab555a886>

In [14]:
def set_obj_ids(x, node_uid, verify_key):
    if hasattr(x, "__dict__") and isinstance(x, SyftObject):
        for prop, val in x.__dict__.items():
            if isinstance(val, (list, tuple)):
                for v in val:
                    set_obj_ids(v, node_uid, verify_key)
            elif isinstance(val, dict):
                for v in val.values():
                    set_obj_ids(v, node_uid, verify_key)
            else:
                set_obj_ids(val, node_uid, verify_key)
        print("setting for", type(x))
        x.syft_node_location=node_uid
        x.syft_client_verify_key=verify_key
        if hasattr(x, "node_uid"):
            print("setting node_UID", type(x))
            x.node_uid=node_uid
                    
                    
    # TODO: hadnle nested collections
            

In [15]:
def sync_items(items, node, client):
    for item in items_to_sync:
        stash = get_stash_for_item(item, node)
#         from copy import deepcopy
#         item = deepcopy(_item)
        
        set_obj_ids(item, client.id, client.verify_key)
#         item.syft_node_location = node.python_node.id
            
#         item.syft_client_verify_key = client.verify_key
        
        res = stash.set(node.python_node.verify_key, item)
        if not res.is_ok():
            raise ValueError("")
        else:
            print("created", item)

In [16]:
sync_items(items_to_sync, node_high, client_high)

setting for <class 'syft.types.datetime.DateTime'>
setting for <class 'syft.store.linked_obj.LinkedObject'>
setting node_UID <class 'syft.store.linked_obj.LinkedObject'>
setting for <class 'syft.service.request.request.UserCodeStatusChange'>
setting for <class 'syft.service.request.request.Request'>
setting node_UID <class 'syft.service.request.request.Request'>
created syft.service.request.request.Request
setting for <class 'syft.types.datetime.DateTime'>
setting for <class 'syft.service.code.user_code.UserCode'>
setting node_UID <class 'syft.service.code.user_code.UserCode'>
created syft.service.code.user_code.UserCode


# Run code high and sync back result

In [17]:
client_high._fetch_api(client_high.credentials)

In [18]:
data_high = client_high.datasets[0].assets[0]

In [19]:
job = client_high.code.compute_mean(data=data_high, blocking=False)
display(job)

```python
class Job:
    id: UID = 8e1ec3d9d9534d63af2b7d3745473ca3
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [20]:
# job.wait().get()

In [21]:
log = job._get_log_objs()

In [22]:
items_to_sync = [job, log]

In [23]:
# job.node_uid

In [24]:
# log.node_uid

In [29]:
client_low.notifications[0]

If possible try using client.requests/client.projects


syft.service.notification.notifications.Notification

In [25]:
x = client_high.requests[0]

In [26]:
x.approve()

Approving request for domain test_h


SyftError: Exception calling request.apply. Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/node/node.py", line 1193, in handle_api_call_with_unsigned_result
    result = method(context, *api_call.args, **api_call.kwargs)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/service.py", line 352, in _decorator
    result = func(self, *args, **kwargs)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/request/request_service.py", line 258, in apply
    mark_as_read(context=context, uid=request_notification.id)
AttributeError: 'NoneType' object has no attribute 'id'

In [29]:
request.node_uid

<UID: 8a1c04544655402190588aec30079bc3>

In [30]:
request.syft_client_verify_key

d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d

In [30]:
# request

In [36]:
client_high.id

<UID: 8212e6797fde4c3fba4fc53ab555a886>

In [37]:
client_high.verify_key

d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d

In [32]:
# set_obj_ids(request, client_high.id, client_high.verify_key)

# x.changes[0].linked_obj.node_uid

# self.changes.linked_obj



In [33]:
from syft.client.api import APIRegistry

In [34]:
for x in APIRegistry.__api_registry__.keys():
    print(x)

(<UID: 8a1c04544655402190588aec30079bc3>, 4597fe5ad2c3c55b169e0d26efd4d08835b22aead82cb4aad0eacca53427a98b)
(<UID: 8a1c04544655402190588aec30079bc3>, 1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367)
(<UID: 8212e6797fde4c3fba4fc53ab555a886>, 0c8a0571187c53f7738eb8b9b25c579628550dab60828ee5012f0e84438a4ddb)
(<UID: 8212e6797fde4c3fba4fc53ab555a886>, d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d)
(<UID: a0e5c2693b8a4100b5c6fa1f4f4430f0>, de402e83e3e40bd2533d68241aacf05000157fa05854421182c5c279fb067a34)
(<UID: a0e5c2693b8a4100b5c6fa1f4f4430f0>, 5f1d600eba827d3394c7260aa736e51f9c7b6ac704b08c0e55c9e404e449acd1)


In [71]:
usercode=list(node_high.python_node.get_service("usercodeservice").stash.partition.data.values())[0]

In [70]:
userco.output_policy

In [25]:
sync_items(items_to_sync, node_low, client_low)

created syft.service.job.job_stash.Job
created syft.service.log.log.SyftLog


# Run code low

In [50]:
res_low = client_low.code.compute_mean(data=data_low)

In [ ]:
# we need to sync the usercode itself


In [52]:
res_low.get()

7.0

In [48]:
private_high.mean()

17.0

In [53]:
res = client_low.code.compute_mean(data=data_low, blocking=False)

SyftWarning: There are existing jobs for this user code, returning the latest one

In [54]:
res

```python
class Job:
    id: UID = 2970d40145144fa4a7aa32182f426cef
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [42]:
res

```python
TwinPointer(Real)
```
7.0

In [55]:
job.result

```python
Pointer
```
```python
class ObjectNotReady:
  id: str = 5f3c2030877d422bb338912bd7bd771b

```

In [56]:
res.wait()

AttributeError: 'str' object has no attribute 'syft_action_data'

In [57]:
%debug

> /Users/koen/workspace/PySyft/packages/syft/src/syft/service/job/job_stash.py(443)wait()
    441                     self.result.id, resolve_nested=False
    442                 )
--> 443                 if isinstance(result_obj.syft_action_data, ActionDataLink) and job_only:
    444                     print(
    445                         "You're trying to wait on a job that has a link as a result."

ipdb> result
*** NameError: name 'result' is not defined
ipdb> print(result_obj)
Could not find item with uid 90d5afc5e82c4517a80364b18892b55f, "90d5afc5e82c4517a80364b18892b55f not in <class 'syft.store.sqlite_document_store.SQLiteBackingStore'>"
ipdb> q
